In [1]:
import geopandas as gpd
import population as ppl
import pandas as pd
import random
import bisect
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.special import ndtr
import numpy as np

# Lectura y preparción de datos

In [35]:
blocks = pd.read_csv('data/Censo2017_Manzanas.csv', encoding='utf-8', sep=';')
blocks = blocks[blocks['COMUNA'] == 2101]
frame_viviendas=pd.read_csv("data/viviendas_antofagasta.csv", encoding='utf-8',sep=',',index_col=0)
people = pd.read_csv('data/personas_antofagasta.csv', encoding='utf-8')
fram_households=pd.read_csv("data/hogares_antofagasta.csv", encoding='utf-8',sep=',',index_col=0)
shape_manzana=gpd.read_file('data/MANZANA_IND_C17.shp')
shape_manzana_geo=gpd.read_file('data/Hogares_georef.shp')

In [36]:
shape_manzana_geo

,OBJECTID,DIRECCIONE,DIRECCIO_1,CALLE,NUMERO,COMPLEMENT,ROL_AVALUO,LATITUD,LONGITUD,REGION,...,LOC_ZON,ENT_MAN,CATEGORIA,NOM_CATEGO,MANZENT_I,TOTAL_PERS,TOTAL_VIVI,SHAPE_Leng,SHAPE_Area,geometry
0,44693.0,ANTOFAGASTA,ANTOFAGASTA,OFICINA LASTARRIA,0,None,0,"-23,550918705","-70,392722875",2,...,17.0,2.0,1.0,CD,2101011017002,1582.0,751.0,0.036910,3.688090e-06,POINT (357848 7394801)
1,44692.0,ANTOFAGASTA,ANTOFAGASTA,OFICINA FRANCISCO PUELMA,0,None,0,"-23,55067767","-70,392125142",2,...,17.0,2.0,1.0,CD,2101011017002,1582.0,751.0,0.036910,3.688090e-06,POINT (357909 7394828)
2,44691.0,ANTOFAGASTA,ANTOFAGASTA,OFICINA LASTARRIA,0,None,0,"-23,550175455","-70,392980752",2,...,17.0,2.0,1.0,CD,2101011017002,1582.0,751.0,0.036910,3.688090e-06,POINT (357821 7394883)
3,44690.0,ANTOFAGASTA,ANTOFAGASTA,OFICINA LASTARRIA,0,None,0,"-23,549621969","-70,392614677",2,...,17.0,2.0,1.0,CD,2101011017002,1582.0,751.0,0.036910,3.688090e-06,POINT (357857 7394945)
4,74683.0,ANTOFAGASTA,ANTOFAGASTA,OFICINA BONA SORT,0,None,0,"-23,548896567","-70,392590837",2,...,17.0,2.0,1.0,CD,2101011017002,1582.0,751.0,0.036910,3.688090e-06,POINT (357859 7395025)
5,44695.0,ANTOFAGASTA,ANTOFAGASTA,SIN NOMBRE,0,None,0,"-23,549736393","-70,394673456",2,...,17.0,5.0,1.0,CD,2101011017005,990.0,399.0,0.018070,2.091800e-06,POINT (357647 7394930)
6,44694.0,ANTOFAGASTA,ANTOFAGASTA,SIN NOMBRE,0,None,0,"-23,549084103","-70,394001022",2,...,17.0,5.0,1.0,CD,2101011017005,990.0,399.0,0.018070,2.091800e-06,POINT (357715 7395003)
7,74681.0,ANTOFAGASTA,ANTOFAGASTA,SIN NOMBRE,0,None,0,"-23,548941735","-70,394583777",2,...,17.0,5.0,1.0,CD,2101011017005,990.0,399.0,0.018070,2.091800e-06,POINT (357656 7395018)
8,45663.0,ANTOFAGASTA,ANTOFAGASTA,FLUORITA,0,None,0,"-23,550632689","-70,391048823",2,...,16.0,19.0,1.0,CD,2101011016019,19.0,6.0,0.008849,4.850570e-06,POINT (358018 7394834)
9,45677.0,ANTOFAGASTA,ANTOFAGASTA,QUEBRADA DE LA CHIMBA,0,None,0,"-23,549572222","-70,390383107",2,...,16.0,19.0,1.0,CD,2101011016019,19.0,6.0,0.008849,4.850570e-06,POINT (358085 7394953)


# Función probabilidades etarias/sexo y de tamaños de hogar para zona

In [4]:
def Size_household(id_zona,frame_viviendas):
    frame_viviendas=frame_viviendas.loc[frame_viviendas['ID_ZONA_LOC']==id_zona]
    sizes_householding=list(frame_viviendas['CANT_PER'])
    try:
        sizes_householding.remove(0)
    except:
        pass
    x1 = np.array(sizes_householding)
    x_eval = np.linspace(1, 10, num=10)
    size_household_cdf = ndtr(np.subtract.outer(x_eval, x1)).mean(axis=1)
    return(size_household_cdf)

In [5]:
def Prob_sex_age(id_zona,blocks,people):
    zone = id_zona
    zone_blocks = blocks[blocks['ID_ZONA_LOC'] == zone]
    zone_people = people[people['ID_ZONA_LOC'] == zone]

    def categorize_age(age):
       if age <= 5:
           return 'EDAD_0A5'
       elif 6 <= age <= 14:
           return 'EDAD_6A14'
       elif 15 <= age <= 64:
           return 'EDAD_15A64'
       else:
           return 'EDAD_65YMAS'

    zone_people['age_range'] = zone_people['P09'].apply(categorize_age)
    zone_counts = zone_people.pivot_table(index='P08', columns='age_range', values='Unnamed: 0', aggfunc='count').fillna(0)
    zone_probs = zone_counts / zone_counts.sum().sum()
#     print("primer print= ",zone_probs)
#     zone_probs = zone_probs[['EDAD_0A5', 'EDAD_6A14','EDAD_15A64','EDAD_65YMAS']]
#     print("segundo print= ",zone_probs)
    age_sex_cdf=[]
    i,j,cum=0,0,0
    for i in range(len(zone_probs.values)):
        for j in zone_probs.columns:
            cum+=zone_probs[j][i+1]
            age_sex_cdf.append(cum)
    return(age_sex_cdf,zone_probs)

# Función generadora de tamaños de hogar y personas

In [6]:
def Size_generator(size_household_cdf,type_household):
    if type_household==1:
        size=0   
    elif type_household==3:
        size=1
    else:
        while True:
            n=random.uniform(0, 1)
            intervals = size_household_cdf
            size=bisect.bisect_left(intervals, n)
            if size>=2:
                break
    return(size+1)

In [7]:
def Age_sex_generator(age_sex_cdf):
    n=random.uniform(0, 1)
    if 0<=n<age_sex_cdf[0]:
        sex=1
        age_min=0
        age_max=5
    elif age_sex_cdf[0]<=n<age_sex_cdf[1]:
        sex=1
        age_min=6
        age_max=14
    elif age_sex_cdf[1]<=n<age_sex_cdf[2]:
        sex=1
        age_min=15
        age_max=64
    elif age_sex_cdf[2]<=n<age_sex_cdf[3]:
        sex=1
        age_min=65
        age_max=150
    elif age_sex_cdf[3]<=n<age_sex_cdf[4]:
        sex=2
        age_min=0
        age_max=5
    elif age_sex_cdf[4]<=n<age_sex_cdf[5]:
        sex=2
        age_min=6
        age_max=14
    elif age_sex_cdf[5]<=n<age_sex_cdf[6]:
        sex=2
        age_min=15
        age_max=64
    elif age_sex_cdf[6]<=n<age_sex_cdf[7]:
        sex=2
        age_min=65
        age_max=150
                    
    return(sex,age_min,age_max)

# Buscador persona

In [8]:
def Search_person(sex,age_min,age_max,persons):
    position=0
    for x in persons:
        if x.sex == sex and age_min<=x.age and x.age<=age_max:
            person_id=x.person_id   
            break
        else:
            person_id = None
        position+=1    
#     next((x for x in persons if x.sex == sex and x.age>=age_min and x.age<=age_max), None)
    return(person_id,position,x)

In [9]:
def Search_person_2(age_min,age_max,persons):
    position=0
    for x in persons:
        if age_min<=x.age and x.age<=age_max:
            person_id=x.person_id   
            break
        else:
            person_id = None
        position+=1    
#     next((x for x in persons if x.sex == sex and x.age>=age_min and x.age<=age_max), None)
    return(person_id,position,x)

In [10]:
def Search_head_household(persons):
    position=0
    for x in persons:
        if 25<=x.age and x.age<=69:
            person_id=x.person_id   
            break
        else:
            person_id = None
        position+=1    
#     next((x for x in persons if x.sex == sex and x.age>=age_min and x.age<=age_max), None)
    return(person_id,position,x)

In [11]:
def Search_couple(person_id,persons_in_block,persons):
    for x in persons:
        if x.person_id==person_id:
            sex=x.sex
            age=x.age
#             education=x.education
            break
#     if education==98 or education==99:  
#         education=7
    if sex==1:
        sex_couple=2
    else:
        sex_couple=1
    position=0    
    for x in persons_in_block:
        if x.sex==sex_couple and x.age>=(age-7) and x.age<=(age+7):
            couple_id=x.person_id
            couple_age=x.age
            couple_sex=x.sex
            break
        else:
            couple_id=None
            x=None
        position+=1    
    return(couple_id,position,x)        

# Genera cantidad de personas por rango etario en block

In [12]:
#PArece que esto no es necesario y esta de mas
def persons_for_agerange(blocks,id_block):
    persons_for_agerange=[]
    range_names=["EDAD_0A5","EDAD_6A14","EDAD_15A64","EDAD_65YMAS"]
    for i in range_names:
        persons_for_agerange.append(blocks.loc[blocks['ID_MANZENT']==id_block][i].item())
    for i in range(len(persons_for_agerange)):
        try:
            persons_for_agerange[i]=int(persons_for_agerange[i])
        except:
            pass
    return(persons_for_agerange)

# Genera personas por block

In [13]:
def persons_for_block(persons_2,id_block,zone_probs,blocks,persons):
#     print("largo persons_2 inicial= ",len(persons_2))
    number_of_persons=blocks.loc[blocks['ID_MANZENT']==id_block]['PERSONAS'].item()  #Obtengo cuanta gente realmente hay en el block
#     print(number_of_persons)
    range_names=["EDAD_0A5","EDAD_6A14","EDAD_15A64","EDAD_65YMAS"] 
    persons_in_block=[]   
    for i in range_names:
        if i=="EDAD_0A5":
            age_min,age_max=0,5
        elif i=="EDAD_6A14":
            age_min,age_max=6,14
        elif i=="EDAD_15A64":
            age_min,age_max=15,64
        elif i=="EDAD_65YMAS":
            age_min,age_max=65,150   
        number=blocks.loc[blocks['ID_MANZENT']==id_block][i].item()
#         try:  #Si el numero de personas en el rango es diferente a * el try va a funcionar y se van a sacar la cantidad de personas dichas
        if number!='*':
            number=int(number)
#             print("NUMBER: ",number)
            for j in range(number):
                try:
#                     print("iteracion= ",j)
#                     print("Edades a buscar= ", age_min,",",age_max)
                    person_id,position,x=Search_person_2(age_min,age_max,persons_2)
#                     print("position= ", position)
#                     print("largo de personas_2= ",len(persons_2))
#                     print("person_id",person_id)
                    del persons_2[position] 
                    persons_in_block.append(x)
                except:
                    pass
        else:       
#         except: #En caso de que sea * se va a generar un número de personas de forma probabilistica segun el cdf de la zona.
            number=int(sum(list(zone_probs[i]))*number_of_persons)
#             print("simule el NUMBER")
#             print("NUMBER: ",number)
            for j in range(number):
                try:
#                     print("Edades a buscar= ", age_min,",",age_max)
                    person_id,position,x=Search_person_2(age_min,age_max,persons_2)   
#                     print("position= ", position)
#                     print("largo de personas_2= ",len(persons_2)) 
#                     print("person_id",person_id)
                    del persons_2[position]
                    persons_in_block.append(x)
                except:
                    pass
#         #Con los Ids busco a sus respectivos objetos
#         persons_in_block_2=[]
#         for i in persons_in_block:
#             for x in persons:
#                 if x.person_id
#     print("largo persons_2 final= ",len(persons_2))  
    return(persons_in_block)

In [14]:
# persons_for_block(persons_2,block,zone_probs,blocks,persons)

# Agregador de personas

In [15]:
def select_person(function,age_min,age_max,persons_in_block,block_households,data_final):
    if function=="Search_person_2":
        global person_id
        person_id,position,member=Search_person_2(age_min,age_max,persons_in_block) #Busco jefe de hogar
    elif function=="Search_couple":
        person_id,position,member=Search_couple(person_id,persons_in_block,persons)
#     print("position= ",position)
#     print("person_id= ",person_id)
#     print("member= ", member)
    if person_id==None:
        hola=0 #No sirve de nada esto
#         print("NO HAY PERSONA ENTRE ",age_min," Y ",age_max)
#         print(persons_in_block)
    else: 
       
        block_households[i].add_member(member) #lo agrego al objeto hogar
        house_id=block_households[i].hh_id
        for x in persons_in_block:
            if x.person_id==person_id:
                age_2=x.age
        del persons_in_block[position]  #lo saco del conjunto de personas por asignar
        #Agrego sus datos al dataframe      
        new_row=pd.DataFrame({'Person_id':person_id, 'Age':age_2, 'Household_id':house_id,'Block_id':block,'Zone_id':id_zona} , index =[0])
        data_final= pd.concat([new_row, data_final]).reset_index(drop = True)
    return(data_final)      


In [32]:
id_zone = list(dict.fromkeys(list(people['ID_ZONA_LOC'])))
i=0
for id_zona in id_zone:
    block=list(blocks.loc[blocks['ID_ZONA_LOC']==id_zona]['ID_MANZENT'])
    for x in block:
        i+=1
print(i)        

3214


# Main

In [16]:
import timeit
start = timeit.timeit()
id_zone = list(dict.fromkeys(list(people['ID_ZONA_LOC'])))
# id_zone=[11932]
# id_zone=[13833,2531,10061]
id_zone=[13833,2531]
data_final=pd.DataFrame(columns=('Person_id', 'Age', 'Household_id','Block_id','Zone_id'))
for id_zona in id_zone:
    print("INICIO ZONA: ",id_zona )
    
    #Genero todas las personas de la zona
    zone_people = people[people['ID_ZONA_LOC'] == id_zona]
    persons = []
    for index, row in zone_people.iterrows():
        persons.append(ppl.Person(row['P09'], row['P08'], row['P07'], row['P15'], row['P18']))
    persons_2=persons.copy() 

    #Genero todos los hogares en la zona
    zone_household=fram_households[fram_households['ID_ZONA_LOC']==id_zona]
    households = []
    for index, row in zone_household.iterrows():
        households.append(ppl.Household(row['TIPO_HOGAR']))
    households_2=households.copy()    

    #Genero el cdf de los tamaños de hogar, sexo y edad en la zona
    size_household_cdf=Size_household(id_zona,frame_viviendas)
    age_sex_cdf,zone_probs=Prob_sex_age(id_zona,blocks,people)
    
    #Empiezo una iteracion por cada manzana
    block=list(blocks.loc[blocks['ID_ZONA_LOC']==id_zona]['ID_MANZENT'])
    for block in block:
    #     input("APRETA ENTER PARA CONTINUAR")
#         print("----------------------------------------------------")
#         print("----------------------------------------------------")
#         print("\n \n INICIO NUEVO BLOCK= ", block)
        #Genero conjunto de personas que existen la manzana
#         print("persons_2= ",len(persons_2))
        persons_in_block=persons_for_block(persons_2,block,zone_probs,blocks,persons)

        #Genero hogares en la manzana
        num_household=int(blocks.loc[blocks['ID_MANZENT']==block]['CANT_HOG'])
        block_households=[]
        for i in range(num_household):
            block_households.append(households_2.pop(random.randint(0,len(households_2)-1)))
        for x in block_households:
            x.add_block_id(block)

        #Comienzo una iteración por cada hogar de una manzana
#         print("numero de casas= ",num_household)
        # create_household_block(num_household,block_households,persons_in_block,size_household_cdf)
        for i in range(num_household):
#             print("\n INICIO CASA NUEVA")
    #         print("person in block= ",persons_in_block)    
#             print("iterador i: ",i)    
            type_household=block_households[i].type
#             print("tipo de casa: ",type_household)
            if type_household==1: #Hogar de una sola personas
                try:
                    data_final=select_person("Search_person_2",27,69,persons_in_block,block_households,data_final) #Función para buscar una persona
                except:
                    pass
            elif type_household==3: #hogar de una pareja
                try:
                    data_final=select_person("Search_person_2",27,69,persons_in_block,block_households,data_final)
                    data_final=select_person("Search_couple",27,69,persons_in_block,block_households,data_final)#Funcion para buscar una pareja de la persona anterior
                except:
                    pass
            elif type_household==2: #Hogar de un padre con hijos
                size_household=Size_generator(size_household_cdf,2)-1 #Genero un tamaño de hogar
#                 print("Tamaño del hogar: ",size_household)
                try:
                    data_final=select_person("Search_person_2",27,69,persons_in_block,block_households,data_final)
                except:
                    pass
                for j in range(size_household-1): 
                    try:
                        data_final=select_person("Search_person_2",0,26,persons_in_block,block_households,data_final) #Busco hijos para la persona
                    except:
                        pass
            else:     #Hogar de multiples personas
                size_household=Size_generator(size_household_cdf,2)-1 #Genero un tamaño de hogar
#                 print("Tamaño del hogar: ",size_household)
                ages=[(28,69),(28,69),(0,18),(0,18),(19,27),(70,150),(28,69),(0,27),(0,27),(70,150)] #Será el orden en que se agrega la gente
                for j in range(size_household): #Voy agregando personas 
                    try:
#                         print("AGREGO PERSONA NUEVA EDAD ENTRE ",ages[j])
                        if j==0:
                            data_final=select_person("Search_person_2",ages[j][0],ages[j][1],persons_in_block,block_households,data_final)
                        if j==1:
                            data_final=select_person("Search_couple",ages[j][0],ages[j][1],persons_in_block,block_households,data_final)
                        else:
                            data_final=select_person("Search_person_2",ages[j][0],ages[j][1],persons_in_block,block_households,data_final)   
                    except:
                        pass
#             print("personas en la casa= ",block_households[i].members)  
#         print("\n person in block SOBRANTES= ",persons_in_block) 
        #Agrego nuevamente las personas sobrantes a al conjunto de personas en la zona
        persons_2.extend(persons_in_block)

end = timeit.timeit()
print ("tiempo de ejecución= ",end - start)


INICIO ZONA:  13833


C:\Users\ggalv\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


INICIO ZONA:  2531
tiempo de ejecución=  -0.0001410844845751132


In [17]:
for x in persons_in_block:
    print(x.person_id)

8247
5109
5214


In [18]:
ages=[x.age for x in persons_2]
ages.sort()
print(ages)
print(len(ages))
import collections
ids=[x.person_id for x in persons_2]
counter=collections.Counter(ids)
print(counter)

[9, 9, 10, 10, 12, 12, 13, 13, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 29, 30, 32, 35, 36, 38, 39, 40, 41, 41, 42, 43, 43, 43, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 52, 59, 65, 65, 66, 67, 67, 67, 67, 67, 67, 67, 67, 69, 69, 69, 69, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 72, 72, 72, 72, 72, 72, 72, 72, 73, 7

In [19]:
#Reviso si hay gente clonada
import collections
lista=[]
for i in range(len(block_households)):
    for x in block_households[i].members:
        lista.append(x.person_id)
counter=collections.Counter(lista)
print(counter)

Counter({8249: 1, 8250: 1, 8251: 1, 8255: 1, 8256: 1, 8241: 1, 8242: 1, 8238: 1, 8239: 1, 8240: 1, 8258: 1, 8259: 1, 8260: 1, 8252: 1, 8262: 1})


In [20]:
#Reviso las edades por cada hogar
data=[]
for i in range(len(block_households)):
    print("CASA NUEVA",i+1," DE TIPO ", block_households[i].type," CON ID ",block_households[i].hh_id )
    for x in block_households[i].members:
        print("ID_Persona= ",x.person_id," Edad= ",x.age, "Sexo= ",x.sex)
        aux=[x.person_id,x.age,x.sex,block_households[i].hh_id]
        data.append(aux)

frame_personas = pd.DataFrame(data, columns = ['Person ID', 'Age','Sex',"House ID"])
frame_personas.to_csv('personas.csv')

CASA NUEVA 1  DE TIPO  5  CON ID  1822
ID_Persona=  8249  Edad=  58 Sexo=  1
ID_Persona=  8250  Edad=  51 Sexo=  1
ID_Persona=  8251  Edad=  48 Sexo=  2
CASA NUEVA 2  DE TIPO  1  CON ID  2307
ID_Persona=  8255  Edad=  64 Sexo=  1
CASA NUEVA 3  DE TIPO  2  CON ID  2365
ID_Persona=  8256  Edad=  62 Sexo=  2
ID_Persona=  8241  Edad=  9 Sexo=  2
ID_Persona=  8242  Edad=  7 Sexo=  1
ID_Persona=  8238  Edad=  26 Sexo=  2
ID_Persona=  8239  Edad=  22 Sexo=  2
ID_Persona=  8240  Edad=  20 Sexo=  2
CASA NUEVA 4  DE TIPO  7  CON ID  1425
ID_Persona=  8258  Edad=  29 Sexo=  1
ID_Persona=  8259  Edad=  48 Sexo=  1
ID_Persona=  8260  Edad=  44 Sexo=  2
ID_Persona=  8252  Edad=  15 Sexo=  1
CASA NUEVA 5  DE TIPO  6  CON ID  1795
ID_Persona=  8262  Edad=  16 Sexo=  2
CASA NUEVA 6  DE TIPO  4  CON ID  2190


In [21]:
for x in block_households:
    if x.hh_id==291:
        print(x.block_id)

In [22]:
#Reviso las edades de la gente sobrante
if len(persons_in_block)==0:
    print("no sobraron personas")
else:    
    for x in persons_in_block:
        print("ID_Persona= ",x.person_id," Edad= ",x.age)

ID_Persona=  8247  Edad=  19
ID_Persona=  5109  Edad=  73
ID_Persona=  5214  Edad=  78


In [26]:
data_final

,Person_id,Age,Household_id,Block_id,Zone_id
0,8262,16,1795,2101011002901,2531
1,8252,15,1425,2101011002901,2531
2,8260,44,1425,2101011002901,2531
3,8259,48,1425,2101011002901,2531
4,8258,29,1425,2101011002901,2531
5,8240,20,2365,2101011002901,2531
6,8239,22,2365,2101011002901,2531
7,8238,26,2365,2101011002901,2531
8,8242,7,2365,2101011002901,2531
9,8241,9,2365,2101011002901,2531


# Partes que pueden ser utiles

In [24]:
for i in range(105811):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-24-eca05154dd56>, line 2)